# Mean-Variance Optimisation

In [1]:
# Import the required libraries and packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import itertools
import math
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from datetime import datetime
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

C:\Users\HP\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# read the datasets into the jupyter notebook

fund_prices = pd.read_csv(r'C:\Users\HP\Python University\fund_prices (version 1).csv')
axa_mmf_ = pd.read_csv(r'C:\Users\HP\Python University\axa_mmf1.csv')

In [3]:
fund_prices['Date'] = pd.to_datetime(fund_prices.Date, format = '%Y-%m-%d') 
axa_mmf_['Date'] = pd.to_datetime(axa_mmf_.Date, format = '%Y-%m-%d')

In [4]:
axa_mmf_['AXA Money Market Fund Yield'] = (1 + axa_mmf_['AXA Money Market Fund Yield']) ** (1/365) - 1

In [5]:
axa_mmf_['month year'] = axa_mmf_['Date'].apply(lambda x: x.strftime('%Y-%m'))
axa_mmf_ = axa_mmf_.drop(columns = ['Date'], axis = 1)

In [6]:
axa_mmf_ = axa_mmf_.groupby('month year')['AXA Money Market Fund Yield'].sum().reset_index()

### Monthly Returns

In [7]:
fund_prices = fund_prices.copy()

In [8]:
# extract some date information

fund_prices['year'] = fund_prices['Date'].dt.year
fund_prices['month'] = fund_prices['Date'].dt.month
fund_prices['day'] = fund_prices['Date'].dt.day
fund_prices['day of month'] = fund_prices['Date'].dt.daysinmonth

In [9]:
first_day28 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 28)]
last_day28 = fund_prices[(fund_prices['day'] == 28) & (fund_prices['day of month'] == 28)]
first_day30 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 30)]
last_day30 = fund_prices[(fund_prices['day'] == 30) & (fund_prices['day of month'] == 30)]
first_day31 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 31)]
last_day31 = fund_prices[(fund_prices['day'] == 31) & (fund_prices['day of month'] == 31)]

In [10]:
new_prices = pd.concat([first_day28, last_day28, first_day30, last_day30, 
                        first_day31, last_day31], axis = 0)

In [11]:
new_prices['month year'] = new_prices['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [12]:
new_prices = new_prices.sort_index(ascending = True)

In [13]:
stb_etf_returns = new_prices['Stanbic ETF 30'].div(new_prices.groupby('month year')['Stanbic ETF 30'].shift(1)) - 1
stb_abs_returns = new_prices['Stanbic Absolute Fund'].div(new_prices.groupby('month year')['Stanbic Absolute Fund'].shift(1)) - 1
stb_gtd_returns = new_prices['Stanbic Guaranteed Income Fund'].div(new_prices.groupby('month year')['Stanbic Guaranteed Income Fund'].shift(1)) - 1
stb_euro_returns = new_prices['Stanbic Eurobond Fund'].div(new_prices.groupby('month year')['Stanbic Eurobond Fund'].shift(1)) - 1
fbn_euro_returns = new_prices['FBN Eurobond Fund'].div(new_prices.groupby('month year')['FBN Eurobond Fund'].shift(1)) - 1

In [14]:
monthly_returns = pd.DataFrame()

monthly_returns['Date'] = new_prices['month year']
monthly_returns['Stanbic ETF 30'] = stb_etf_returns
monthly_returns['Stanbic Absolute Fund'] = stb_abs_returns
monthly_returns['Stanbic Guaranteed Income Fund'] = stb_gtd_returns
monthly_returns['Stanbic Eurobond Fund'] = stb_euro_returns
monthly_returns['FBN Eurobond Fund'] = fbn_euro_returns
#monthly_returns['AXA Money Market Fund Yield'] = axa_mmf_['AXA Money Market Fund Yield']

In [15]:
monthly_returns = monthly_returns.dropna()

In [16]:
monthly_returns = monthly_returns.reset_index(drop = True, inplace = False )

In [17]:
month_returns = pd.concat([monthly_returns, axa_mmf_['AXA Money Market Fund Yield']], axis = 1)

### Portfolio Return

In [18]:
returns1 = month_returns[['Stanbic Absolute Fund', 'AXA Money Market Fund Yield']]
returns2 = month_returns[['Stanbic Absolute Fund', 'Stanbic Guaranteed Income Fund', 'AXA Money Market Fund Yield']]            
returns3 = month_returns[['Stanbic Absolute Fund', 'Stanbic ETF 30']]
returns4 = month_returns[['Stanbic Eurobond Fund', 'FBN Eurobond Fund']]

In [19]:
axa_mmf1, axa_mmf2, axa_mmf3 = [0.9, 0.72, 0.13]
stb_abs1, stb_abs2, stb_abs3, stb_abs4, stb_abs5 = [0.1, 0.24, 0.87, 0.73, 0.51]
stb_GI = 0.04
stb_etf1, stb_etf2 = [0.27, 0.49]
stb_euro, fbn_euro = [0.5, 0.5]

In [20]:
conserv = np.array([stb_abs1, axa_mmf1])
caut = np.array([stb_abs2, stb_GI, axa_mmf2])
bal = np.array([stb_abs3, axa_mmf3])
grow = np.array([stb_abs4, stb_etf1])
aggres = np.array([stb_abs5, stb_etf2])
doll = np.array([stb_euro, fbn_euro])

In [21]:
conservative = np.dot(returns1, conserv)
cautious = np.dot(returns2, caut)
balanced = np.dot(returns1, bal)
growth = np.dot(returns3, grow)
aggressive = np.dot(returns3, aggres)
dollar = np.dot(returns4, doll)

In [22]:
portfolios = pd.DataFrame()

portfolios['Date'] = month_returns['Date']
portfolios['Conservative'] = conservative
portfolios['Cautious'] = cautious
portfolios['Balanced'] = balanced
portfolios['Growth'] = growth
portfolios['Aggressive'] = aggressive
portfolios['Dollar'] = dollar

In [ ]:
portfolios.head()

## Regression

In [ ]:
col_names = ['Oil Prices', 'Inflation', 'NSE ASI', 'FX Reserves', 'USD/NGN (BDC)', 
             'USD/NGN (I&E)', '91-Day Yield', '182-Day Yield', '364-Day Yield', 
             '2yr-Bond Yield', '3yr-Bond Yield', '5-yr-Bond Yield', '7yr-Bond Yield', 
             '10yr-Bond Yield', '12yr-Bond Yield', '15yr-Bond Yield', '20yr-Bond Yield']

macro_data = pd.read_csv(r'C:\Users\HP\Python University\Portfolios & Macro Data.csv', usecols = col_names)

In [ ]:
data = pd.concat([portfolios, macro_data], axis = 1)

In [ ]:
data.columns

Check the correlation each macro data variable with the  conservative portfolio

#### Conservative

In [ ]:
# Independent Variables
X1 = data[['Oil Prices', 'Inflation', 'NSE ASI', 'FX Reserves',
       'USD/NGN (BDC)', 'USD/NGN (I&E)', '91-Day Yield', '182-Day Yield',
       '364-Day Yield', '2yr-Bond Yield', '3yr-Bond Yield', '5-yr-Bond Yield',
       '7yr-Bond Yield', '10yr-Bond Yield', '12yr-Bond Yield',
       '15yr-Bond Yield', '20yr-Bond Yield']]

# Dependent Variable
Y1 = data['Conservative']

In [ ]:
X1a = sm.add_constant(X1)
reg1 = sm.OLS(Y1, X1a).fit()
#prediction1 = reg1.predict(X1)

reg1.summary()

In [ ]:
# Independent Variables
X1_conserv = data[['NSE ASI', 'FX Reserves']]

# Dependent Variable
Y1_conserv = data['Conservative']

X1a_conserv = sm.add_constant(X1_conserv)
reg1_conserv = sm.OLS(Y1_conserv, X1a_conserv).fit()
#prediction1 = reg1.predict(X1)

reg1_conserv.summary()

### Cautious

In [ ]:
Y2 = data['Cautious']

X1b = sm.add_constant(X1)
reg2 = sm.OLS(Y2, X1b).fit()
#prediction1 = reg1.predict(X1)

reg2.summary()

In [ ]:
# Independent Variables
X1_caut = data[['NSE ASI', 'FX Reserves']]

# Dependent Variable
Y2_caut = data['Cautious']


X1b_caut = sm.add_constant(X1_caut)
reg2_caut = sm.OLS(Y2_caut, X1b_caut).fit()
#prediction1 = reg1.predict(X1)

reg2_caut.summary()

### Balanced

In [ ]:
Y3 = data['Balanced']


X1c = sm.add_constant(X1)
reg3 = sm.OLS(Y3, X1c).fit()
#prediction1 = reg1.predict(X1)

reg3.summary()

In [ ]:
# Independent Variables
X1_bal = data[['NSE ASI', 'USD/NGN (BDC)']]

# Dependent Variable
Y2_bal = data['Balanced']


X1b_bal = sm.add_constant(X1_bal)
reg2_bal = sm.OLS(Y2_bal, X1b_bal).fit()
#prediction1 = reg1.predict(X1)

reg2_bal.summary()

### Growth

In [ ]:
Y4 = data['Growth']


X1d = sm.add_constant(X1)
reg4 = sm.OLS(Y4, X1d).fit()
#prediction1 = reg1.predict(X1)

reg4.summary()

### Aggressive

In [ ]:
Y5 = data['Aggressive']


X1e = sm.add_constant(X1)
reg5 = sm.OLS(Y5, X1e).fit()
#prediction1 = reg1.predict(X1)

reg5.summary()

### Dollar

In [ ]:
Y6 = data['Dollar']


X1f = sm.add_constant(X1)
reg6 = sm.OLS(Y6, X1f).fit()
#prediction1 = reg1.predict(X1)

reg6.summary()

### Monte-Carlo Simulation

In [ ]:
stb_etf_returns1 = (new_prices['Stanbic ETF 30'].div(new_prices.groupby('month year')['Stanbic ETF 30'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_abs_returns1 = (new_prices['Stanbic Absolute Fund'].div(new_prices.groupby('month year')['Stanbic Absolute Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_gtd_returns1 = (new_prices['Stanbic Guaranteed Income Fund'].div(new_prices.groupby('month year')['Stanbic Guaranteed Income Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_euro_returns1 = (new_prices['Stanbic Eurobond Fund'].div(new_prices.groupby('month year')['Stanbic Eurobond Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
fbn_euro_returns1 = (new_prices['FBN Eurobond Fund'].div(new_prices.groupby('month year')['FBN Eurobond Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
axa_mmf_returns1 = axa_mmf_['AXA Money Market Fund Yield'].reset_index(drop = True)

In [ ]:
month_returns_log = pd.DataFrame({'Date': month_returns['Date'], 
                                  'Stanbic ETF 30': stb_etf_returns1, 
                                  'Stanbic Absolute Fund': stb_abs_returns1, 
                                  'Stanbic Guaranteed Income Fund': stb_gtd_returns1, 
                                  'Stanbic Eurobond Fund': stb_euro_returns1, 
                                  'FBN Eurobond Fund': fbn_euro_returns1, 
                                  'AXA Money Market Fund Yield': axa_mmf_returns1})                                 

In [ ]:
stb_etf_mean = np.array(month_returns_log['Stanbic ETF 30'].mean())
stb_abs_mean = np.array(month_returns_log['Stanbic Absolute Fund'].mean())
stb_gtb_mean = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].mean())
stb_euro_mean = np.array(month_returns_log['Stanbic Eurobond Fund'].mean())
fbn_euro_mean = np.array(month_returns_log['FBN Eurobond Fund'].mean())
axa_mmf_mean = np.array(month_returns_log['AXA Money Market Fund Yield'].mean())

In [ ]:
stb_etf_var = np.array(month_returns_log['Stanbic ETF 30'].var())
stb_abs_var = np.array(month_returns_log['Stanbic Absolute Fund'].var())
stb_gtb_var = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].var())
stb_euro_var = np.array(month_returns_log['Stanbic Eurobond Fund'].var())
fbn_euro_var = np.array(month_returns_log['FBN Eurobond Fund'].var())
axa_mmf_var = np.array(month_returns_log['AXA Money Market Fund Yield'].var())

In [ ]:
stb_etf_std = np.array(month_returns_log['Stanbic ETF 30'].std())
stb_abs_std = np.array(month_returns_log['Stanbic Absolute Fund'].std())
stb_gtb_std = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].std())
stb_euro_std = np.array(month_returns_log['Stanbic Eurobond Fund'].std())
fbn_euro_std = np.array(month_returns_log['FBN Eurobond Fund'].std())
axa_mmf_std = np.array(month_returns_log['AXA Money Market Fund Yield'].std())

In [ ]:
stb_etf_drift = np.array(stb_etf_mean - ( 0.5 * stb_etf_var))
stb_abs_drift = np.array(stb_abs_mean - ( 0.5 * stb_abs_var))
stb_gtb_drift = np.array(stb_gtb_mean - ( 0.5 * stb_gtb_var))
stb_euro_drift = np.array(stb_euro_mean - ( 0.5 * stb_euro_var))
fbn_euro_drift = np.array(fbn_euro_mean - ( 0.5 * fbn_euro_var))
axa_mmf_drift = np.array(axa_mmf_mean - ( 0.5 * axa_mmf_var))

In [ ]:
norm.ppf(0.95)

In [ ]:
x = np.random.rand(10, 2)
x

In [ ]:
norm.ppf(x)

In [ ]:
z = norm.ppf(np.random.rand(10, 2))
z

In [ ]:
t_intervals = 1200
n_iterations = 1000

In [ ]:
stb_etf_pred = np.exp(stb_etf_drift + stb_etf_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
stb_abs_pred = np.exp(stb_abs_drift + stb_abs_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
stb_gtb_pred = np.exp(stb_gtb_drift + stb_gtb_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
stb_euro_pred = np.exp(stb_euro_drift + stb_euro_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
fbn_euro_pred = np.exp(fbn_euro_drift + fbn_euro_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
axa_mmf_pred = np.exp(axa_mmf_drift + axa_mmf_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))

In [ ]:
stb_etf_returns_list = np.zeros_like(stb_etf_pred)
stb_abs_returns_list = np.zeros_like(stb_abs_pred)
stb_gtb_returns_list = np.zeros_like(stb_gtb_pred)
stb_euro_returns_list = np.zeros_like(stb_euro_pred)
fbn_euro_returns_list = np.zeros_like(fbn_euro_pred)
axa_mmf_returns_list = np.zeros_like(axa_mmf_pred)

In [ ]:
stb_etf_returns_list[0] = month_returns_log['Stanbic ETF 30'].iloc[-1]
stb_abs_returns_list[0] = month_returns_log['Stanbic Absolute Fund'].iloc[-1]
stb_gtb_returns_list[0] = month_returns_log['Stanbic Guaranteed Income Fund'].iloc[-1]
stb_euro_returns_list[0] = month_returns_log['Stanbic Eurobond Fund'].iloc[-1]
fbn_euro_returns_list[0] = month_returns_log['FBN Eurobond Fund'].iloc[-1]
axa_mmf_returns_list[0] = month_returns_log['AXA Money Market Fund Yield'].iloc[-1]

In [ ]:
for t in range(1, t_intervals):
    stb_etf_returns_list[t] = stb_etf_returns_list[t - 1] * stb_etf_pred[t]
    stb_abs_returns_list[t] = stb_abs_returns_list[t - 1] * stb_abs_pred[t]
    stb_gtb_returns_list[t] = stb_gtb_returns_list[t - 1] * stb_gtb_pred[t]
    stb_euro_returns_list[t] = stb_euro_returns_list[t - 1] * stb_euro_pred[t]
    fbn_euro_returns_list[t] = fbn_euro_returns_list[t - 1] * fbn_euro_pred[t]
    axa_mmf_returns_list[t] = axa_mmf_returns_list[t - 1] * axa_mmf_pred[t]

In [ ]:
stb_etf_returns_list

In [ ]:
axa_mmf1, axa_mmf2, axa_mmf3 = [0.9, 0.72, 0.13]
stb_abs1, stb_abs2, stb_abs3, stb_abs4, stb_abs5 = [0.1, 0.24, 0.87, 0.73, 0.51]
stb_GI = 0.04
stb_etf1, stb_etf2 = [0.27, 0.49]
fbn_euro, stb_euro = [0.5, 0.5]

In [ ]:
future_dates = pd.date_range(start = '2018-12', periods = 1200, freq = 'M')
future_dates.strftime('%Y-%m')
index = 0

#### Conservative Portfolio Simulation

In [ ]:
stb_a = stb_abs_returns_list * stb_abs1
axa_a = axa_mmf_returns_list * axa_mmf1

conserv = np.sum([stb_a, axa_a], axis = 0)
conservative = np.round(100000 + (conserv * 100000), 3)

conservative = pd.DataFrame(data = conservative[:, :])
conservative.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

conservative.head()

#### Cautious Portfolio Simulation

In [ ]:
stb_b = stb_abs_returns_list * stb_abs2
stb_g = stb_gtb_returns_list * stb_GI
axa_b = axa_mmf_returns_list * axa_mmf2

caut = np.sum([stb_b, stb_g, axa_b], axis = 0)
cautious = np.round(100000 + (caut * 100000), 3)

cautious = pd.DataFrame(data = cautious[:, :])
cautious.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

cautious.tail()

#### Balanced Portfolio Simulation

In [ ]:
stb_c = stb_abs_returns_list * stb_abs3
axa_c = axa_mmf_returns_list * axa_mmf3

bal = np.sum([stb_c, axa_c], axis = 0)
balanced = np.round(100000 + (bal * 100000), 3)

balanced = pd.DataFrame(data = balanced[:, :])
balanced.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

balanced.head()

#### Growth Portfolio Simulation

In [ ]:
stb_d1 = stb_abs_returns_list * stb_abs4
stb_d2 = stb_etf_returns_list * stb_etf1

grow = np.sum([stb_d1, stb_d2], axis = 0)
growth = np.round(100000 + (grow * 100000), 3)

growth = pd.DataFrame(data = growth[:, :])
growth.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

growth.head()

#### Aggressive Portfolio Simulation

In [ ]:
stb_e1 = stb_abs_returns_list * stb_abs5
stb_e2 = stb_etf_returns_list * stb_etf2

aggres = np.sum([stb_e1, stb_e2], axis = 0)
aggressive = np.round(100000 + (aggres * 100000), 3)

aggressive = pd.DataFrame(data = aggressive[:, :])
aggressive.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

aggressive.head()

#### Dollar Portfolio Simulation

In [ ]:
stb_f = stb_euro_returns_list * stb_euro
fbn_a = fbn_euro_returns_list * fbn_euro

doll = np.sum([stb_f, fbn_a], axis = 0)
dollar = np.round(100000 + (doll * 100000), 3)

dollar = pd.DataFrame(data = dollar[:, :])
dollar.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

dollar.head()

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
stb_gtb_mean, stb_gtb_std

In [ ]:
A = pd.DataFrame(data = stb_etf_returns_list[:, :])

A = round((A * 100), 3)
A